## Evaluate imputation panels

### prepare datasets

#### multiple imps (UKB)

In [ ]:
discovery="PGC2_noAJ"
target="dbg-scz19"
imp_pops="eur,sas,gbr"
imp_pops=(${imp_pops//,/ })
base_rep="105"
n_folds=5
pheno=ht # TBD Probably redundant. Check it!
## Adjust reps and folds according to a parameter

#Prepare inner CV (train+validation)
for imp_pop in imp_pops; do 
    for cur_rep in {1..6}; do 
        for cur_fold in {1..5}; do 
            bash prepare_cv_datasets.sh --discovery ${discovery} --target ${target} \
            --imp impute2_1kg_${imp_pop} --rep ${base_rep}_${cur_rep} \
            --cv ${cur_fold}_${n_folds} --pheno ${pheno}; 
        done 
    done
done

#Prepare outer CV (both+test)
for imp_pop in imp_pops; do 
    for cur_rep in {1..6}; do 
        bash prepare_cv_datasets.sh --discovery ${discovery} --target ${target} \
        --imp impute2_1kg_${imp_pop} --rep ${base_rep}_${cur_rep} \
        --cv ${n_folds} --pheno ${pheno} 
    done 
done


### Generate pheno files to subsets defined for the nested CV (```prepare_datsets.sh```)

In [ ]:

for cur_rep in {1..6}; do 
    cd rep_105_${cur_rep}
    for pheno in chol t2d gerx madd angna ast ctrt gerx height hfvr hyty osar utfi; do 
        echo "rep_105_${cur_rep} ${pheno}"
        ls -1 pheno_ht_* | xargs -I A bash -c 'fl=$(echo A | awk '"'"'{ gsub(/_ht_/, "_'${pheno}'_"); print $0}'"'"'); awk '"'"'{if(NR==FNR){c[$1]++; next}; if (c[$1]>0){print $0}}'"'"' <(cat A) <(cat ../pheno_'${pheno}'_) > ${fl}'
    done
    cd ..
done

### Run CV across folds

#### Generally the best practive it os run the scripts ```run_folds/run_UKB_folds_<method>.sh```. The run only parts that are missing in the CV scheme and is generally faster and safer.

In [ ]:
method=ls

discoveries="D2_sysp_evangelou_2018,D2_dias_evangelou_2018,D2_asth_zhu_2019,D2_chol_willer_2013,D2_ldlp_willer_2013,D2_t2di_mahajan_2018,D2_gerx_an_2019,D2_madd_howard_2019" # "D2_sysp_evangelou_2018" D2_hdlp_willer_2013
# discoveries="UKB_ht_eur,UKB_chol_eur,UKB_hfvr_eur,UKB_hyty_eur,UKB_madd_eur,UKB_osar_eur,UKB_t2d_eur,UKB_utfi_eur,UKB_gerx_eur,UKB_angna_eur,UKB_ast_eur,UKB_ctrt_eur"
targets="ukbb_afr"
imps="original" # "impute2_1kg_gbr" # ,impute2_1kg_afr,impute2_1kg_gbr" # "impute2_1kg_eur,impute2_1kg_afr,impute2_1kg_gbr,impute2_1kg_sas" # "impute2_1kg_ceu2"

bash run_folds_${method}.sh --discoveries=${discoveries} --targets=${targets} --imps=${imps}

#### To override existing results, use the scripts below:

#### Run PRS method across reps

In [ ]:
discovery="PGC2_noAJ"
target="dbg-scz19"
imp="impute2_ajkg14_t101" # "impute2_1kg_ceu2"
method="ld"
base_rep=105

for cur_rep in {1..6}; do 
    bash run_cv_single_rep.sh --discovery ${discovery} --target ${target} --imp ${imp} \
    --method ${method} --rep ${base_rep}_${cur_rep} --start_fold 2 && \
    bash run_cv_test_single_rep.sh --discovery ${discovery} --target ${target} --imp ${imp} \
    --method ${method} --rep ${base_rep}_${cur_rep}; 
done

#### Altentively, it can be executed as two separated commads (usefule for reruns of failed executions)

In [ ]:
discovery="PGC2_noAJ"
target="dbg-scz19"
imp="impute2_ajkg14_t101" # "impute2_1kg_ceu2"
method="ld"
base_rep=105

for cur_rep in {1..6}; do 
    bash run_cv_single_rep.sh --discovery ${discovery} --target ${target} --imp ${imp} \
    --method ${method} --rep ${base_rep}_${cur_rep}
done

for cur_rep in {1..6}; do 
    bash run_cv_test_single_rep.sh --discovery ${discovery} --target ${target} --imp ${imp} \
    --method ${method} --rep ${base_rep}_${cur_rep}
done


### Clean lasso output weight files from 0 weighted SNPs

In [ ]:
cat $PRS_CODEBASE/lasso_folders | head -n 5 | xargs -I {} bash -c 'ls -d {}*' | \
 xargs -I A -n 1 -P 80  bash -c  'echo "start A" && cat A | awk '"'"'{if($6!=0){print $0}}'"'"' > "A_" && sleep 0 | mv A_ A && echo "done A"'